In [ ]:
from heatmap_model.interaction_model import UQnet, TrajModel
import numpy as np
import sys
import matplotlib.pyplot as plt
%matplotlib inline 

import torch
from torch import nn, Tensor
import torchvision.datasets as dataset
from torch.optim.lr_scheduler import StepLR
import datetime

from scipy.special import expit

from heatmap_model.utils import *
from heatmap_model.train import *
from heatmap_model.interaction_dataset import *
from heatmap_model.inference import *
from heatmap_model.losses import *
from config import *

from absl import logging
logging._warn_preinit_stderr = 0
logging.warning('Worrying Stuff')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

In [ ]:
# change hyper-parameters here
paralist['resolution'] = 0.5
paralist['encoder_attention_size'] = 128
paralist['use_sem'] = False
paralist['epoches'] = 6
paralist['mode'] = 'lanescore'
paralist['prob_mode'] = 'ce'

In [ ]:
model = UQnet(paralist, test=True).to(device)
trajmodel = TrajModel().to(device)
#testset = InteractionDataset(['val'], 'val', paralist, mode=paralist['mode'], filters=True)
testset = InteractionDataset(['test'], 'test', paralist, mode=paralist['mode'], filters=False)

In [ ]:
Yp, H = InferenceModel(model, './interactionDE/', 1, testset, paralist)
H.shape, Yp.shape

In [ ]:
np.savez_compressed('./results/prediction_d', yp=Yp, heatmap=H)

In [ ]:
data = np.load('./results/prediction_d.npz', allow_pickle=True)
Yp = data['yp']
H = data['heatmap']

In [ ]:
Generate_csv(trajmodel, './interactionDE/', Yp)